Google Colab Setup

In [ ]:
# Clone the GitHub repository to the local environment
!git clone https://github.com/jan1na/Bleeding-Detecting-on-Capsule-Endoscopy.git

# Import the sys module to modify the Python path
import sys

# Add the "scripts" directory to the Python path so that Python can access the modules in that directory
sys.path.append('/content/Bleeding-Detecting-on-Capsule-Endoscopy/scripts')


Drive Setup

In [ ]:
from google.colab import drive

# Mount Google Drive (for data storage and access to dataset)
drive.mount('/content/drive')

# Deep Learning Part

## Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.optim import lr_scheduler
from torch.utils.data.sampler import WeightedRandomSampler

from models import MobileNetV2
from bleeding_dataset import BleedDataset

## Training Space

### Hyperparameters

In [ ]:
# Define the path where model checkpoints and results will be saved
SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/runs/"

# Define the train/test split ratio
TRAIN_TEST_SPLIT = (0.8, 0.1)  # remaining parts will be used for testing

# Set the directory path for the dataset
DIRECTORY_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/project_capsule_dataset"

# Set the batch size for training
BATCH_SIZE = 16

# Define the learning rate for the optimizer
LR = 0.001  # learning rate

# Number of epochs to train the model
NUM_OF_EPOCHS = 20

# Early stopping limit: number of epochs with no improvement before stopping
EARLY_STOP_LIMIT = 3

# Threshold for predicting bleeding or healthy: values above this threshold are considered bleeding
THRESHOLD = 0.5  # predictions above this threshold will be considered as bleeding

# Select the model architecture: MobileNetV2, ResNet, AlexNet, or VGG19
MODEL = MobileNetV2  # Options: ResNet, AlexNet, VGG19

# Whether to apply data augmentation during training
APPLY_AUGMENTATION = False

# Whether to use cosine annealing for learning rate scheduling
USE_COSINE_ANNEALING_LR = True

# Number of times to augment each image during training
AUGMENT_TIMES = 8


### Dataset, Model etc. Inıtıalization

In [ ]:
# Set the device for training (CUDA if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()  # Clear any cached memory on the GPU (if using CUDA)


# Model Initialization
def initialize_model(model_class, save_path: str):
    """
    
    
    :param model_class: 
    :param save_path: 
    :return: 
    """
    # Initialize the model class and move it to the appropriate device (GPU/CPU)
    model = model_class().to(device)

    # Generate a serial number based on the model name and current timestamp for saving
    model_serial_number = f"training_with_{model.__class__.__name__}_{datetime.now().strftime('on_%m.%d._at_%H:%M:%S')}"

    # Set the path where the model will be saved during training
    model_serial_path = os.path.join(save_path, model_serial_number)

    # Create the directory for saving the model if it doesn't exist
    os.makedirs(model_serial_path, exist_ok=True)

    return model, model_serial_path

# Initialize the model and get the save path for training
model, model_serial_path = initialize_model(MODEL, SAVE_PATH)

# Placeholder for the full dataset (initialized later)
full_dataset = None


# Dataset Preparation
def prepare_datasets(dataset_class, directory: str, split_ratios: list[int], batch_size: int, image_mode: str = "RGB", seed: int = 0):
    """
    
    
    :param dataset_class: 
    :param directory: 
    :param split_ratios: 
    :param batch_size: 
    :param image_mode: 
    :param seed: 
    :return: 
    """
    global full_dataset
    # Load the full dataset with augmentation settings
    full_dataset = dataset_class(directory, mode=image_mode, apply_augmentation=APPLY_AUGMENTATION,
                                 augment_times=AUGMENT_TIMES)

    # Calculate the sizes of the train, test, and validation sets
    total_size = len(full_dataset)
    train_size = int(split_ratios[0] * total_size)
    test_size = int(split_ratios[1] * total_size)
    validation_size = total_size - train_size - test_size

    # Set the random seed for reproducibility
    torch.manual_seed(seed)

    # Split the dataset into train, validation, and test sets
    train_dataset, validation_dataset, test_dataset = random_split(full_dataset,
                                                                   [train_size, validation_size, test_size])

    # Oversample the bleeding class in the training set (for class imbalance handling)
    train_labels = np.array(full_dataset.get_labels())[train_dataset.indices]
    class_counts = np.bincount(train_labels)  # Count occurrences of each class (0 = healthy, 1 = bleeding)
    class_weights = 1.0 / class_counts  # Compute inverse class frequencies

    # Calculate weights for each sample based on its label
    weights = [class_weights[label] for label in train_labels]

    # Create a sampler to enforce the class weights during training
    sampler = WeightedRandomSampler(weights, len(weights))

    # Return the DataLoaders for the train, validation, and test datasets
    return {
        "train": DataLoader(train_dataset, batch_size=batch_size, num_workers=4, pin_memory=True, sampler=sampler),
        "validation": DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=4,
                                 pin_memory=True),
        "test": DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
    }

# Prepare the datasets and generate DataLoader objects for train, validation, and test sets
data_loaders = prepare_datasets(BleedDataset, DIRECTORY_PATH, TRAIN_TEST_SPLIT, BATCH_SIZE, image_mode="RGB", seed=0)

# Calculate the weight for the bleeding class to handle class imbalance
bleeding_weight = 6161 / (713 * AUGMENT_TIMES) if APPLY_AUGMENTATION else 6161 / 713
weights = torch.tensor([1.0, bleeding_weight]).to(device)

# Define the loss function with a positive weight for the bleeding class
criterion = nn.BCEWithLogitsLoss(pos_weight=weights[1])

# Initialize the optimizer for training (Adam optimizer)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Choose the learning rate scheduler based on whether cosine annealing is enabled
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_OF_EPOCHS) if USE_COSINE_ANNEALING_LR else lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


### Training Loop

In [ ]:
### ---|---|---|---|---|---|---|---|---|---|--- TRAINING ---|---|---|---|---|---|---|---|---|---|--- ###

# Lists to keep track of the training and validation losses over epochs
train_losses, validation_losses = [], []

# Variables for early stopping (to stop training if the validation loss does not improve)
min_validation_loss = None  # minimum achieved loss on validation dataset, used for early stopping
min_validation_path = None  # path to model checkpoint file
early_stop_step = 0  # number of epochs since the last improvement in validation loss

# Loop over the number of epochs
for epoch in range(NUM_OF_EPOCHS):
    # Enable data augmentation if specified
    if APPLY_AUGMENTATION:
        full_dataset.enable_augmentation()

    averaged_training_loss = 0  # variable to track the cumulative training loss for this epoch

    # Training loop over batches of the training dataset
    for batch_idx, (images, labels) in tqdm(enumerate(data_loaders['train']), leave=False):
        images, labels = images.to(device), labels.to(device)  # move data to the appropriate device (GPU/CPU)

        model = model.train()  # set model to training mode
        outputs = model(images.type(torch.float))  # get model outputs (predictions)

        # Extract the predictions and labels for loss calculation
        float_outputs = outputs[:, 0].type(torch.float)
        float_labels = labels.type(torch.float)

        # Calculate the training loss using BCEWithLogitsLoss
        train_loss = criterion(float_outputs, float_labels)
        averaged_training_loss = averaged_training_loss + train_loss  # accumulate the loss

        optimizer.zero_grad()  # clear previous gradients
        train_loss.backward()  # compute gradients for backpropagation
        optimizer.step()  # update model weights

    # Calculate the average training loss for this epoch
    averaged_training_loss = averaged_training_loss / len(data_loaders['train'])

    # Validation loop (no gradients needed here, so use torch.no_grad())
    with torch.no_grad():
        if APPLY_AUGMENTATION:
            full_dataset.disable_augmentation()  # turn off augmentation during validation

        model = model.eval()  # set model to evaluation mode

        # Calculate validation loss
        validation_loss = 0.0
        for validation_images, validation_labels in data_loaders['validation']:
            validation_images, validation_labels = validation_images.to(device), validation_labels.to(device)

            # Get the model's predictions for the validation data
            validation_outputs = model(validation_images.type(torch.float))

            # Extract the predictions and labels for loss calculation
            float_validation_outputs = validation_outputs[:, 0].type(torch.float)
            float_validation_labels = validation_labels.type(torch.float)

            # Add the loss for this batch to the total validation loss
            validation_loss += criterion(float_validation_outputs, float_validation_labels).item()

        # Average the validation loss over the entire validation set
        validation_loss /= len(data_loaders['validation'])

    # If this is the first validation or a new minimum validation loss is achieved
    if min_validation_loss is None or validation_loss < min_validation_loss:
        early_stop_step = 0  # reset early stopping counter
        min_validation_loss = validation_loss  # update minimum validation loss

        # If there is a checkpoint from the previous best model, remove it
        if min_validation_path is not None:
            os.remove(min_validation_path)

        # Save the current model as a checkpoint with the new minimum validation loss
        min_validation_path = os.path.join(model_serial_path,
                                           "min_validation_loss:" + str(min_validation_loss) + "_epoch:" + str(
                                               epoch) + ".pth")
        torch.save(model, min_validation_path)
    else:
        # Increment early stopping counter if validation loss did not improve
        early_stop_step += 1

    # Log the losses and append them to the respective lists
    print(
        f"Epoch: {epoch + 1} | training loss: {averaged_training_loss.item()} | min validation loss: {min_validation_loss}",
        flush=True)
    train_losses.append(averaged_training_loss.item())
    validation_losses.append(validation_loss)

    # Adjust the learning rate based on the scheduler
    scheduler.step()

    # Check if early stopping should be triggered (if validation loss hasn't improved for several epochs)
    if early_stop_step >= EARLY_STOP_LIMIT:
        print("early stopping...")
        break


In [ ]:
plt.plot(train_losses, color='blue', label='Train Loss')
plt.plot(validation_losses, color='orange', label='Validation Loss')
plt.legend()
plt.savefig(os.path.join(model_serial_path, "losses.png"))

## Testing Space

In [ ]:
### ---|---|---|---|---|---|---|---|---|---|--- TESTING ---|---|---|---|---|---|---|---|---|---|--- ###
loaded_model = torch.load(min_validation_path)
loaded_model = loaded_model.eval()

# class_correct counts how many correct predictions for that label [corrects_for_label_0, corrects_for_label_1]
# class_total counts how many predictions are there for that label [predictions_for_label_0, predictions_for_label_1]
class_correct, class_total = [0, 0], [0, 0]
with torch.no_grad():
    if APPLY_AUGMENTATION:
        full_dataset.disable_augmentation()
    for test_images, test_labels in tqdm(data_loaders['test']):
        test_images, test_labels = test_images.to(device), test_labels.to(device)

        test_outputs = loaded_model(test_images.type(torch.float))

        test_outputs = test_outputs.squeeze().type(torch.float)
        test_outputs[test_outputs >= THRESHOLD] = 1
        test_outputs[test_outputs < THRESHOLD] = 0

        # calculate indices for correct predictions
        correct = (test_outputs == test_labels).squeeze()
        for e, label in enumerate(test_labels):
            # increase the correct prediction count for that label
            class_correct[label] += correct[e].item()
            class_total[label] += 1

In [ ]:
# Total: accuracy for whole dataset
# Print out the total accuracy for the entire dataset by summing up the correct predictions and dividing by the total number of samples
print(f"Total accuracy: {sum(class_correct) / sum(class_total)} on threshold: {THRESHOLD}")

# Print accuracy for healthy detection (class 0)
print(f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0] / class_total[0]}")

# Print accuracy for bleeding detection (class 1)
print(f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1] / class_total[1]}")

# Save the accuracy results to a text file in the model's directory
with open(os.path.join(model_serial_path, "accuracy.txt"), 'w') as txt:
    # Write total accuracy to the file
    txt.write(f"Total accuracy: {sum(class_correct) / sum(class_total)} on threshold: {THRESHOLD}\n")

    # Write healthy detection accuracy to the file
    txt.write(f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0] / class_total[0]}\n")

    # Write bleeding detection accuracy to the file
    txt.write(f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1] / class_total[1]}\n")

# Clear the GPU memory cache after evaluation to avoid memory overflow
torch.cuda.empty_cache()
